In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt
import torch.nn.init as init

In [2]:
def x_train_data(N_x, x_l, x_r, del_x, N_bc):

    x_train = np.linspace(x_l, x_r, N_x)
    x_bc1 = np.zeros(N_bc)
    x_bc2 = np.ones(N_bc)*x_r
    x_ic = np.
    x_train = np.concatenate((x_train,x_bc1,x_bc2),0)
    x_train = torch.FloatTensor(x_train)
    x_train = x_train.unsqueeze(-1)
    x_train = x_train.clone().detach().requires_grad_(True)
    
    return x_train

def t_train_data(N_tot, t_curr):
    
    t_train = np.ones(N_tot)*t_curr
    t_train = torch.FloatTensor(t_train)
    t_train = t_train.unsqueeze(-1)
#     t_train = t_train.clone().detach().requires_grad_(True)
    
    return t_train


def Initial_Cond(N_x, T_l):
    T_ini = np.zeros(N_x)
    T_ini[0] = T_l
    T_ini = torch.FloatTensor(T_ini)
    T_ini = T_ini.unsqueeze(-1)
#     T_ini = T_ini.clone().detach().requires_grad_(True)
    
    return T_ini

def RK_coeffs(q):
    
    if q == 4:
        coeff = np.array([1/6,1/3,1/3,1/6])
        
    coeff = torch.FloatTensor(coeff)
    coeff = coeff.unsqueeze(-1)
    coeff = coeff.clone().detach().requires_grad_(True)
        
    return coeff
            

def xavier_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0.1)
    
class FVMANN(nn.Module):
    def __init__(self, layer_size):
        super(FVMANN, self).__init__()
        
        # Fully conected model
        modules = []
        for i in range(len(layer_size) - 1):
            modules.append(nn.Linear(layer_size[i], layer_size[i+1]))  
            modules.append(nn.Tanh())

        self.fc = nn.Sequential(*modules)
        self.fc.apply(xavier_init)
        
    def forward(self, x_train, t_train):
        op = self.fc( torch.cat((x_train, t_train), 1) )
        op_x = torch.autograd.grad(op, x_train, grad_outputs=torch.ones_like(op), create_graph=True)[0]
        op_x2 = torch.autograd.grad(op_x, x_train, grad_outputs=torch.ones_like(op_x), create_graph=True)[0]
        return op, op_x, op_x2
    
def train_model(model, optimiser, epochs, T_r, T_l, k1, N_x, x_l, x_r, del_x, t_i, t_f, del_t, N_t, N_bc, q):
    
    loss_store = []
    mse = nn.MSELoss(reduction='sum')
    model.train()  
    
    # Loss function Weights
    w1 = 1
    w2 = 1
    w3 = 1
        
    # Null array
    N_tot = N_x + 2*N_bc
    null = torch.zeros(N_tot)
    null = null.unsqueeze(-1)
    
    # Generate training points and coefficients
    x_train = x_train_data(N_x, x_l, x_r, del_x, N_bc)
    T_ini = Initial_Cond(N_x, T_l)
    coeff = RK_coeffs(q)

    for epoch in range(epochs):
        for j in range(N_t):
            
            t_train = t_train_data(N_tot, t_i + del_t*(j))
#             print(t_train)
            
            T, dTdx, d2Tdx2 = model(x_train, t_train)
            T_new = T[:,q].unsqueeze(-1)
            T_stage = T[0:N_x,:-1]
            
            if j==0:
                eq1 = w1*mse(T_new[0:N_x] , T_ini)/(N_x)
            else:
                eq1 = w1*mse((T_new[0:N_x] - T_old)/del_t, k1*torch.matmul(T_stage, coeff))/(N_x)
                
            bc1 = w2*mse( torch.mul(torch.where(x_train == x_l,1,0),(T_new - T_l)), null )/(N_bc + 1)
            bc2 = w3*mse( torch.mul(torch.where(x_train == x_r,1,0),(T_new - T_r)), null )/(N_bc + 1)

            optimiser.zero_grad()
            loss = eq1 + bc1 + bc2
            loss.backward()
            optimiser.step()

            loss_store.append(loss.detach().numpy())
            
            T_old = T_new[0:N_x].detach().clone()
        
            if j%5==0:   
                print('epoch = ',j)
                print('loss = ',loss.detach().numpy())
                print('eq1_loss = ',eq1.detach().numpy())
                print('bc1_loss = ',bc1.detach().numpy())
                print('bc2_loss = ',bc2.detach().numpy())

    return loss_store

In [3]:
N_x = 40
N_bc = 50
x_l = 0
x_r = 0.7
t_i = 0
t_f = 0.3
N_t = 30
del_t = (t_f - t_i)/(N_t - 1)
del_x = (x_r - x_l)/(N_x - 1)
T_l = 1.0
T_r = 0.0
t_range = [0, 0.2, 0.6, 0.8]

# Neural network params
q = 4 # R-K stages
layer_size = [2, 45, 45, 45, 45, q+1]

# material params
k1 = 1

# Training data and initial data
model = FVMANN(layer_size)
print(model)
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable parameters in the model:", total_trainable_params)

# Setup Loss function and Optimiser
learning_rate = 6e-6
optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)
epochs = 40000

# Training model
loss_store = train_model(model, optimiser, epochs, T_r, T_l, k1, N_x, x_l, x_r, del_x, t_i, t_f, del_t, N_t, N_bc, q)

FVMANN(
  (fc): Sequential(
    (0): Linear(in_features=2, out_features=45, bias=True)
    (1): Tanh()
    (2): Linear(in_features=45, out_features=45, bias=True)
    (3): Tanh()
    (4): Linear(in_features=45, out_features=45, bias=True)
    (5): Tanh()
    (6): Linear(in_features=45, out_features=45, bias=True)
    (7): Tanh()
    (8): Linear(in_features=45, out_features=5, bias=True)
    (9): Tanh()
  )
)
Total trainable parameters in the model: 6575
epoch =  0
loss =  1.418962
eq1_loss =  0.025905412
bc1_loss =  1.3846111
bc2_loss =  0.008445559
epoch =  5
loss =  1.5186007
eq1_loss =  0.098020814
bc1_loss =  1.4103882
bc2_loss =  0.010191639
epoch =  10
loss =  1.5431919
eq1_loss =  0.09556347
bc1_loss =  1.4356784
bc2_loss =  0.011949995
epoch =  15
loss =  1.5685713
eq1_loss =  0.09419663
bc1_loss =  1.4605802
bc2_loss =  0.013794465
epoch =  20
loss =  1.5935152
eq1_loss =  0.092705965
bc1_loss =  1.485087
bc2_loss =  0.015722191
epoch =  25
loss =  1.6176573
eq1_loss =  0.0908

loss =  1.0167557
eq1_loss =  0.08639758
bc1_loss =  0.88536453
bc2_loss =  0.044993598
epoch =  0
loss =  0.8876649
eq1_loss =  0.042349216
bc1_loss =  0.77805495
bc2_loss =  0.06726075
epoch =  5
loss =  0.9375664
eq1_loss =  0.08132897
bc1_loss =  0.79129684
bc2_loss =  0.064940564
epoch =  10
loss =  0.9498803
eq1_loss =  0.08261468
bc1_loss =  0.8045409
bc2_loss =  0.062724784
epoch =  15
loss =  0.96263057
eq1_loss =  0.084186085
bc1_loss =  0.8178887
bc2_loss =  0.060555838
epoch =  20
loss =  0.9750214
eq1_loss =  0.08531022
bc1_loss =  0.8312797
bc2_loss =  0.0584315
epoch =  25
loss =  0.9866106
eq1_loss =  0.08566601
bc1_loss =  0.8445737
bc2_loss =  0.0563709
epoch =  0
loss =  0.8735343
eq1_loss =  0.05135558
bc1_loss =  0.74211735
bc2_loss =  0.08006136
epoch =  5
loss =  0.912288
eq1_loss =  0.080007456
bc1_loss =  0.7545891
bc2_loss =  0.07769147
epoch =  10
loss =  0.923955
eq1_loss =  0.081454426
bc1_loss =  0.76708484
bc2_loss =  0.075415775
epoch =  15
loss =  0.936

epoch =  10
loss =  0.750757
eq1_loss =  0.06698837
bc1_loss =  0.3864476
bc2_loss =  0.297321
epoch =  15
loss =  0.7572555
eq1_loss =  0.06970974
bc1_loss =  0.39164573
bc2_loss =  0.29590002
epoch =  20
loss =  0.76346874
eq1_loss =  0.07206957
bc1_loss =  0.39700124
bc2_loss =  0.29439798
epoch =  25
loss =  0.7691819
eq1_loss =  0.07388214
bc1_loss =  0.40244034
bc2_loss =  0.29285944
epoch =  0
loss =  0.9199436
eq1_loss =  0.24545014
bc1_loss =  0.3560322
bc2_loss =  0.31846124
epoch =  5
loss =  0.7411798
eq1_loss =  0.06331221
bc1_loss =  0.36047456
bc2_loss =  0.31739306
epoch =  10
loss =  0.74737716
eq1_loss =  0.06607443
bc1_loss =  0.36507958
bc2_loss =  0.31622314
epoch =  15
loss =  0.753649
eq1_loss =  0.06883352
bc1_loss =  0.3698789
bc2_loss =  0.3149366
epoch =  20
loss =  0.75964296
eq1_loss =  0.07124514
bc1_loss =  0.37483704
bc2_loss =  0.31356078
epoch =  25
loss =  0.765151
eq1_loss =  0.07312626
bc1_loss =  0.37988412
bc2_loss =  0.31214064
epoch =  0
loss = 

loss =  0.7750349
eq1_loss =  0.057279278
bc1_loss =  0.15937966
bc2_loss =  0.55837595
epoch =  15
loss =  0.7796803
eq1_loss =  0.060216192
bc1_loss =  0.16067226
bc2_loss =  0.5587919
epoch =  20
loss =  0.78410506
eq1_loss =  0.062969066
bc1_loss =  0.16210091
bc2_loss =  0.5590351
epoch =  25
loss =  0.7882121
eq1_loss =  0.06543287
bc1_loss =  0.16363612
bc2_loss =  0.5591431
epoch =  0
loss =  1.2046635
eq1_loss =  0.48505372
bc1_loss =  0.1481526
bc2_loss =  0.5714572
epoch =  5
loss =  0.7753584
eq1_loss =  0.05396048
bc1_loss =  0.14902702
bc2_loss =  0.5723709
epoch =  10
loss =  0.7800052
eq1_loss =  0.05689202
bc1_loss =  0.15003718
bc2_loss =  0.573076
epoch =  15
loss =  0.7845962
eq1_loss =  0.059823588
bc1_loss =  0.15119411
bc2_loss =  0.57357854
epoch =  20
loss =  0.7889701
eq1_loss =  0.06258115
bc1_loss =  0.15248396
bc2_loss =  0.573905
epoch =  25
loss =  0.79303235
eq1_loss =  0.06506125
bc1_loss =  0.15387918
bc2_loss =  0.5740919
epoch =  0
loss =  1.224536
e

loss =  0.85077214
eq1_loss =  0.050581302
bc1_loss =  0.064741656
bc2_loss =  0.7354492
epoch =  10
loss =  0.8548533
eq1_loss =  0.05324994
bc1_loss =  0.0647761
bc2_loss =  0.73682725
epoch =  15
loss =  0.85885197
eq1_loss =  0.055947933
bc1_loss =  0.06490282
bc2_loss =  0.7380012
epoch =  20
loss =  0.8626725
eq1_loss =  0.058569144
bc1_loss =  0.06511673
bc2_loss =  0.7389866
epoch =  25
loss =  0.8662711
eq1_loss =  0.061058473
bc1_loss =  0.065406926
bc2_loss =  0.7398057
epoch =  0
loss =  1.4704642
eq1_loss =  0.66656816
bc1_loss =  0.061143987
bc2_loss =  0.742752
epoch =  5
loss =  0.855831
eq1_loss =  0.05041509
bc1_loss =  0.061064973
bc2_loss =  0.74435097
epoch =  10
loss =  0.859873
eq1_loss =  0.05305813
bc1_loss =  0.061070208
bc2_loss =  0.74574465
epoch =  15
loss =  0.8638305
eq1_loss =  0.055730015
bc1_loss =  0.061164565
bc2_loss =  0.7469359
epoch =  20
loss =  0.86761534
eq1_loss =  0.058332823
bc1_loss =  0.06134303
bc2_loss =  0.74793947
epoch =  25
loss = 

...

epoch =  10
loss =  1.0000154
eq1_loss =  1.543549e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  15
loss =  1.0000111
eq1_loss =  1.1088623e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  20
loss =  1.0000159
eq1_loss =  1.5837284e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  25
loss =  1.0000275
eq1_loss =  2.7522767e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  0
loss =  1.9359362
eq1_loss =  0.93593615
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  5
loss =  1.0000385
eq1_loss =  3.8495673e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  10
loss =  1.0000141
eq1_loss =  1.4125181e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  15
loss =  1.0000112
eq1_loss =  1.1228057e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  20
loss =  1.0000168
eq1_loss =  1.6836218e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  25
loss =  1.0000312
eq1_loss =  3.128238e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  0
loss =  1.9359548
eq1_loss =  0.93595475
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  5
loss =  1.0000281
eq1_loss =  2.

loss =  1.0000203
eq1_loss =  2.0298865e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  0
loss =  1.936255
eq1_loss =  0.936255
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  5
loss =  1.000094
eq1_loss =  9.4073956e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  10
loss =  1.0000671
eq1_loss =  6.71554e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  15
loss =  1.0000616
eq1_loss =  6.160582e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  20
loss =  1.000115
eq1_loss =  0.00011504871
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  25
loss =  1.0004182
eq1_loss =  0.00041813226
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  0
loss =  1.9362751
eq1_loss =  0.9362751
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  5
loss =  1.0001308
eq1_loss =  0.00013073203
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  10
loss =  1.000013
eq1_loss =  1.2942083e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  15
loss =  1.0000082
eq1_loss =  8.267362e-06
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  20
loss =  1.000011
eq1_loss =  1.1000413e-05
bc1_loss =

epoch =  5
loss =  1.0000309
eq1_loss =  3.0850693e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  10
loss =  1.0000094
eq1_loss =  9.45088e-06
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  15
loss =  1.0000073
eq1_loss =  7.2825865e-06
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  20
loss =  1.0000125
eq1_loss =  1.2541127e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  25
loss =  1.0000201
eq1_loss =  2.0190539e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  0
loss =  1.9366021
eq1_loss =  0.9366022
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  5
loss =  1.0000608
eq1_loss =  6.0775645e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  10
loss =  1.0000175
eq1_loss =  1.7572484e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  15
loss =  1.0000104
eq1_loss =  1.0419931e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  20
loss =  1.000015
eq1_loss =  1.502795e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  25
loss =  1.0000243
eq1_loss =  2.4276325e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  0
loss =  1.9366207
eq1_loss =  0.

loss =  1.0000166
eq1_loss =  1.6536233e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  25
loss =  1.0000322
eq1_loss =  3.222784e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  0
loss =  1.936893
eq1_loss =  0.936893
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  5
loss =  1.0000249
eq1_loss =  2.496798e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  10
loss =  1.0000104
eq1_loss =  1.0403791e-05
bc1_loss =  0.0
bc2_loss =  1.0
epoch =  15
loss =  1.0000094
eq1_loss =  9.459648e-06
bc1_loss =  0.0
bc2_loss =  1.0


KeyboardInterrupt: 

In [ ]:
x_train = train_data(N_x, x_l, x_r, del_x, N_bc)
T, dTdx, d2Tdx2 = model(x_train)
plt.plot(x_train[0:N_x].detach().numpy() ,T[0:N_x,q].detach().numpy())